<a href="https://colab.research.google.com/github/bharathmb7/eBPF-project/blob/main/ebpf_model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import re
import numpy as np

# Read data from the text file
with open('process_monitor_log.txt', 'r') as file:
    lines = file.readlines()

# Parse the data and aggregate based on PID
data = {}
current_pid = None
current_entry = {}
for line in lines:
    if line.strip() == '-------------------------':
        if current_pid is not None:
            if current_pid not in data:
                data[current_pid] = []
            data[current_pid].append(current_entry)
            current_entry = {}
    else:
        key, value = re.split(r':\s*', line.strip(), maxsplit=1)
        if key == 'PID':
            current_pid = int(value)
        else:
            current_entry[key] = value

# Convert data into a dictionary of numpy arrays
aggregated_data = {}
for pid, entries in data.items():
    aggregated_data[pid] = np.array([[entry.get('Read Amount', 0),
                                      entry.get('Write Amount', 0),
                                      entry.get('File read Count', 0),
                                      entry.get('File write Count', 0),
                                      entry.get('File Open Count', 0),
                                      entry.get('File Create Count', 0),
                                      entry.get('File Unlink Count', 0),
                                      entry.get('File Rename Count', 0)]
                                     for entry in entries])

# Now you have preprocessed data aggregated based on PID

In [3]:
len(aggregated_data)

199

In [4]:
aggregated_data

{54728: array([['20', '0', '19', '0', '0', '0', '0', '0'],
        ['20', '0', '19', '1', '0', '0', '0', '0'],
        ['20', '3369', '19', '2', '0', '0', '0', '0']], dtype='<U4'),
 6506: array([['0', '46180681', '2', '5', '0', '0', '0', '0'],
        ['0', '46180681', '2', '5', '0', '0', '0', '0']], dtype='<U8'),
 1547: array([['1010136585', '1084543036', '19', '64', '0', '0', '2', '1'],
        ['0', '2786983305', '1', '649', '0', '0', '0', '0']], dtype='<U10'),
 2690: array([['0', '0', '1', '2', '0', '0', '0', '0'],
        ['0', '0', '7', '2', '0', '0', '0', '0']], dtype='<U1'),
 6505: array([['1518788098', '0', '3', '1', '0', '0', '0', '0'],
        ['2494939713', '2211402196', '32', '469', '0', '0', '0', '0']],
       dtype='<U10'),
 11730: array([['0', '0', '4', '1', '0', '0', '0', '0'],
        ['1630019436', '764756482', '193', '89', '217', '0', '0', '0']],
       dtype='<U10'),
 28815: array([['2016580363', '3486586159', '11', '47', '0', '0', '0', '0'],
        ['1804225292',

In [5]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = aggregated_data

# Preprocess the data
X = []
y = []

# Pad or trim arrays to a fixed length
max_length = max(len(value.flatten()) for value in data.values())
X = []
y = []

for key, value in data.items():
    # Flatten or concatenate the arrays within each element
    flattened = value.flatten()
    # Pad or trim arrays to max_length
    padded = np.pad(flattened, (0, max_length - len(flattened)), mode='constant')
    # Append to X
    X.append(padded)
    # Assign label
    y.append(key)  # Assuming all data is labeled as 1 (ransomware)

for x in X:
  x = x[:3]



In [6]:
y

[54728,
 6506,
 1547,
 2690,
 6505,
 11730,
 28815,
 6510,
 54057,
 1008,
 0,
 2701,
 11684,
 6537,
 1,
 11681,
 55326,
 55424,
 55450,
 54890,
 55627,
 54858,
 55387,
 55713,
 54763,
 55729,
 54918,
 11682,
 55189,
 55370,
 55618,
 55313,
 54826,
 55456,
 55418,
 54754,
 55316,
 54940,
 54926,
 55144,
 55186,
 54871,
 54773,
 55328,
 55664,
 55440,
 55406,
 55547,
 54874,
 54867,
 55251,
 55280,
 54980,
 55407,
 54764,
 55139,
 54828,
 54802,
 55396,
 55283,
 55391,
 55026,
 55345,
 55559,
 55344,
 54765,
 55717,
 54841,
 54824,
 55652,
 55153,
 55262,
 55282,
 54804,
 55403,
 55156,
 54815,
 55443,
 55263,
 54746,
 55579,
 55499,
 54994,
 54760,
 54894,
 55402,
 55322,
 55299,
 55342,
 55360,
 54957,
 55141,
 54830,
 55278,
 1450,
 54902,
 54948,
 55425,
 55554,
 54930,
 55264,
 55725,
 54997,
 55436,
 55546,
 55301,
 54941,
 55295,
 54986,
 54972,
 54991,
 2810,
 54836,
 55457,
 55570,
 54863,
 55380,
 55575,
 54892,
 55256,
 55419,
 55445,
 55606,
 55361,
 12206,
 55287,
 54849,
 5

In [7]:
# Convert lists to arrays
X = np.array(X)
print(X.shape)
y = np.array(y)

# Randomly assign labels (1 for ransomware, 0 for benign)
labels = np.random.choice([0, 1], size=len(y))

# Create DataFrame
df = pd.DataFrame({'Process ID': y, 'Label': labels})

# Display the dataset
print(df)

y = df['Label'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X)
print(y)
print(X.shape)
print(y.shape)


(199, 24)
     Process ID  Label
0         54728      0
1          6506      0
2          1547      0
3          2690      0
4          6505      0
..          ...    ...
194       55434      1
195       54781      1
196       55386      0
197       55451      1
198       55250      1

[199 rows x 2 columns]
[['20' '0' '19' ... '0' '0' '0']
 ['0' '46180681' '2' ... '0' '0' '0']
 ['1010136585' '1084543036' '19' ... '0' '0' '0']
 ...
 ['0' '2886692864' '0' ... '0' '0' '0']
 ['3366650961' '0' '3' ... '0' '0' '0']
 ['0' '990090496' '0' ... '0' '0' '0']]
[0 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 1
 1 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0
 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 0 0 1
 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 1 1 1 0 1 0 0 0 1 1 1
 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1 1
 0 0 1 1 0 1 0 0 1 1 1 0 1 1]
(199, 24)
(199,)


In [8]:

# Train SVM model
svm_classifier = SVC(kernel='linear', random_state=42, verbose=True)



In [ ]:
svm_classifier.fit(X_train, y_train)

[LibSVM]

In [ ]:

# Evaluate on test set
y_pred = svm_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [3]:
# Assign labels to each entry
X = []
y = []
for pid, entries in aggregated_data.items():
    tmp = []
    for entry in entries:
      # tmp.append(entry.astype(int))
      X.append(entry.astype(int))  # Convert to float
      y.append(pid)



In [4]:
print(X)
print(y)

[array([20,  0, 19,  0,  0,  0,  0,  0]), array([20,  0, 19,  1,  0,  0,  0,  0]), array([  20, 3369,   19,    2,    0,    0,    0,    0]), array([       0, 46180681,        2,        5,        0,        0,
              0,        0]), array([       0, 46180681,        2,        5,        0,        0,
              0,        0]), array([1010136585, 1084543036,         19,         64,          0,
                0,          2,          1]), array([         0, 2786983305,          1,        649,          0,
                0,          0,          0]), array([0, 0, 1, 2, 0, 0, 0, 0]), array([0, 0, 7, 2, 0, 0, 0, 0]), array([1518788098,          0,          3,          1,          0,
                0,          0,          0]), array([2494939713, 2211402196,         32,        469,          0,
                0,          0,          0]), array([0, 0, 4, 1, 0, 0, 0, 0]), array([1630019436,  764756482,        193,         89,        217,
                0,          0,          0]), array([20

In [6]:
# Convert lists to numpy arrays
X = np.array(np.array(X))
y = np.array(y)

In [9]:
print(X)
print(y)
print(X.shape)
print(y.shape)

[[        20          0         19 ...          0          0          0]
 [        20          0         19 ...          0          0          0]
 [        20       3369         19 ...          0          0          0]
 ...
 [         0 2886692864          0 ...          0          0          0]
 [3366650961          0          3 ...          0          0          0]
 [         0  990090496          0 ...          0          0          0]]
[54728 54728 54728  6506  6506  1547  1547  2690  2690  6505  6505 11730
 11730 28815 28815  6510  6510 54057 54057  1008  1008     0     0  2701
  2701 11684 11684  6537  6537     1     1 11681 11681 55326 55424 55450
 54890 55627 54858 55387 55713 54763 55729 54918 11682 55189 55370 55618
 55313 54826 55456 55418 54754 55316 54940 54926 55144 55186 54871 54773
 55328 55664 55440 55406 55547 54874 54867 55251 55280 54980 55407 54764
 55139 54828 54802 55396 55283 55391 55026 55345 55559 55344 54765 55717
 54841 54824 55652 55153 55262 55282 54804 55

In [17]:
# Randomly assign labels (1 for ransomware, 0 for benign)
labels = np.random.choice([0, 1], size=len(y))

# Create DataFrame
df = pd.DataFrame({'Process ID': y, 'Label': labels})

# Display the dataset
print(df)

y = df['Label'].values

# X = X[:10]
# y = y[:10]
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

   Process ID  Label
0           0      1
1           0      1
2           0      0
3           0      1
4           0      1
5           0      1
6           1      0
7           0      1
8           0      0
9           0      1


In [18]:
print(X_train)
print(y_train)
print(X_train.shape)
print(y_train.shape)

[[1010136585 1084543036         19         64          0          0
           2          1]
 [        20          0         19          0          0          0
           0          0]
 [         0          0          1          2          0          0
           0          0]
 [        20       3369         19          2          0          0
           0          0]
 [1518788098          0          3          1          0          0
           0          0]
 [         0   46180681          2          5          0          0
           0          0]
 [         0   46180681          2          5          0          0
           0          0]
 [         0 2786983305          1        649          0          0
           0          0]]
[1 1 1 0 1 1 1 0]
(8, 8)
(8,)


In [19]:
# prompt: print x_train shape




In [20]:
# Train SVM
svm_classifier = SVC(kernel='linear', random_state=42, verbose=True)




In [21]:
svm_classifier.fit(X_train, y_train)

[LibSVM]

SVC(kernel='linear', random_state=42, verbose=True)

In [22]:
# Predict
y_pred = svm_classifier.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5
